In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals
from termcolor import cprint
import tensorflow as tf
import IPython.display as display
import numpy as np
import os
import pathlib
import cv2 as cv2
import random

AUTOTUNE = tf.data.experimental.AUTOTUNE

print("Tensorflow Version: ", tf.__version__)
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))


Tensorflow Version:  2.1.0
Num GPUs Available:  1


In [2]:
data_dir = pathlib.Path("data/")

# Get class names from classes.txt
CLASS_NAMES = []
with open(data_dir/"classes.txt") as classes_file:
    CLASS_NAMES = classes_file.read().splitlines()

n_samples = 0
# Count number of mp4 files in data dir
for c in CLASS_NAMES:
    n_samples += len(list(data_dir.glob('**/{}/*.mp4'.format(c))))

print("Samples: {}, Classes: {} {}".format(n_samples, len(CLASS_NAMES), CLASS_NAMES))

Samples: 386, Classes: 2 ['Idle', 'Speak']


In [3]:
# Initialize numpy dataset
dataset = np.zeros(shape=(n_samples, 15, 240, 320, 3), dtype=np.float32)
labels = np.zeros(shape=(n_samples, len(CLASS_NAMES)), dtype=np.uint8)

In [4]:
# Gets the label of a file from the path
def get_label(path):
    return path.parts[-2]


# Load a frame from an opencv capture and returns it as a numpy ndarray
def load_frame_as_ndarray(cap):
    ret, frame = cap.read()
    if ret:
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        frame = np.asarray(frame)
        frame = frame.astype('float32')
        frame *= 1.0/255.0
        return frame
    else:
        return None


# Loads the video file in the provided path as an array of frames
def load_video_as_ndarray(path):
    if path.is_file():
        #print("Loading file {}...".format(path))
        cap = cv2.VideoCapture(str(path.absolute()))
        n_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

        if n_frames > 15:
            cprint("WARNING: Video file {} contains more than 15 frames (was: {}). Extra frames will be ignored.".format(path, n_frames), 'yellow')
        elif n_frames < 15:
            cprint("WARNING: Video file {} contains less than 15 frames (was: {}). Last frame will be duplicated.".format(path, n_frames), 'yellow')

        frames = []
        for i in range(15):
            frame = load_frame_as_ndarray(cap)
            if frame is not None:
                frames.append(frame)
            else:
                frames.append(frames[i-1])

        frames = np.asarray(frames)
        return frames
    else:
        cprint("ERROR: File does not exist '{}'".format(path), 'red')
        return None

# Warning for classes in dataset not declared in classes.txt
undeclared_classes=[]
def warn_undeclared(label):
    if label not in undeclared_classes:
        cprint("WARNING: A file was found with class '{}', not declared in 'classes.txt'. The file will be skipped.".format(label), 'yellow')
        undeclared_classes.append(label)

In [5]:
#Test load video function
#video = load_video_as_ndarray(pathlib.Path("data/14/Examine/000390.mp4"))

In [6]:
n_loaded = 0
files_list = list(data_dir.glob('**/*.mp4'))
random.shuffle(files_list)
for f in files_list:
    label = get_label(f)
    if label in CLASS_NAMES:
        dataset[n_loaded] = load_video_as_ndarray(f)
        labels[n_loaded][CLASS_NAMES.index(label)] = 1
        n_loaded += 1
    else:
        warn_undeclared(label)

In [7]:
# Export dataset to npz file
np.savez('export/data/dataset.npz', dataset=dataset, labels=labels)

In [8]:
exit()